In [1]:
import numpy as np
import pandas as pd
import datetime as dt

In [3]:
# Data cleaning
# limit data from 2004-2005 season to 2021-2022 season since these dont have missing data

In [ ]:
# loading data
players = pd.read_csv("all_seasons.csv", usecols=range(1,22))
games_details = pd.read_csv("games_details.csv")
games = pd.read_csv("games.csv")
teams = pd.read_csv("teams.csv")

In [ ]:
# Cleaning players.csv file to retain players who were part of the San Antonio Spurs at least once

players["year"] = players.season.astype(str).str.split('-', expand=True)[0].astype(int).reset_index(drop=True)
players_filtered = players[(players['year']>=2004) & (players['year']<=2021)].sort_values(by='season').reset_index(drop=True) # keep only players who were in the nba between 2004-2005 season and 2021-2022 season

# create unique list of players who were on the Spurs at least once from 2004-2005 to 2021-2022 season
players_spurs = players_filtered[players_filtered["team_abbreviation"]=='SAS'].drop_duplicates('player_name').reset_index(drop=True)
players_spurs.sort_values('year', ascending=True).reset_index(drop=True)

# retrieve players stats over the years for both current and ex spurs
players_stats = players_filtered[players_filtered['player_name'].isin(players_spurs['player_name'])]

In [ ]:
# adding HOME_TEAM_NAME
games_joined = games.merge(teams[["TEAM_ID","ABBREVIATION"]], how='left', left_on="HOME_TEAM_ID", right_on="TEAM_ID").reset_index(drop=True)
games_joined.rename({"ABBREVIATION": "HOME_TEAM_NAME"}, axis=1, inplace=True)
games_joined.drop(games_joined.columns[-2], axis=1, inplace=True)

# adding AWAY_TEAM_NAME
games_joined = games_joined.merge(teams[["TEAM_ID","ABBREVIATION"]], how='left', left_on="VISITOR_TEAM_ID", right_on="TEAM_ID").reset_index(drop=True)
games_joined.rename({"ABBREVIATION": "AWAY_TEAM_NAME"}, axis=1, inplace=True)
games_joined.drop(games_joined.columns[-2], axis=1, inplace=True)

# adding a column that states the opposing team
games_joined['Opposing_Team'] = games_joined.apply(lambda row: row['HOME_TEAM_NAME'] if row['HOME_TEAM_NAME'] != "SAS" else row['AWAY_TEAM_NAME'], axis=1)

# add the team city
games_joined = games_joined.merge(teams[["ABBREVIATION", "CITY"]], how='left', left_on="Opposing_Team", right_on="ABBREVIATION").reset_index(drop=True)
games_joined.drop(games_joined.columns[-2], axis=1, inplace=True)

# SHOW ONLY GAMES FROM RELEVANT DATES
games_joined['GAME_DATE_EST'] = pd.to_datetime(games_joined['GAME_DATE_EST'])
games_filtered = games_joined[(games_joined['SEASON']>= 2004) & (games_joined['SEASON'] <=2021)]
games_filtered = games_filtered.reset_index(drop=True).sort_values("GAME_DATE_EST")

# Show only games where the spurs played
games_filtered_spurs = games_filtered[(games_filtered['AWAY_TEAM_NAME']=='SAS') | (games_filtered['HOME_TEAM_NAME']=='SAS')]

# adding a spurs win column
spurs_win_condition = [(games_filtered_spurs["HOME_TEAM_WINS"]==1) & (games_filtered_spurs["HOME_TEAM_NAME"]=='SAS'), 
             (games_filtered_spurs["HOME_TEAM_WINS"]==0) & (games_filtered_spurs["AWAY_TEAM_NAME"]=='SAS')] # spurs win condition
spurs_win_values = [1,1]
games_filtered_spurs['SPURS_WON']=np.select(spurs_win_condition, spurs_win_values)

In [ ]:
# spurs team id: 1610612759

# cleaning games_details data

# retrieve only players who played for spurs at some point in their careers
games_details_spurs = games_details[games_details['PLAYER_NAME'].isin(players_spurs["player_name"])].reset_index(drop=True)

# retrieve boxscores of spurs games only
games_details_spurs = games_details_spurs[games_details_spurs['TEAM_ID']==1610612759]

# append game_date_est + season
games_details_spurs = games_details_spurs.merge(games_filtered_spurs[['GAME_DATE_EST', 'GAME_ID', 'SEASON']], on='GAME_ID')

In [33]:
'''
data sets to use: 
games_details_spurs: for player stats per game
games_filtered_spurs: for spurs win loss record and aggregated stats
players_stats: current and ex-spurs stats
'''
# Create a Pandas Excel writer using XlsxWriter as the engine
writer = pd.ExcelWriter('Spurs Data Updated.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet
games_details_spurs.to_excel(writer, sheet_name='games_details_spurs', index=False)
games_filtered_spurs.to_excel(writer, sheet_name='games_filtered_spurs', index=False)
players_stats.to_excel(writer, sheet_name='players_stats', index=False)

# Save the workbook
writer.save()

In [ ]:
# Close the workbook
writer.close()

In [19]:
# Csv with ALL players stats
players_filtered.to_csv('all_players.csv', index=False)  # Set index=False to avoid saving the index